In [14]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import pandapower as pp
from pandapower.networks import case1354pegase

In [15]:
# Run the case file and check if there is any error
net = case1354pegase()

# Set the sgen NOT in service
net.sgen.in_service = False

# Set the maximum and minimum power output of the generator
net.gen.max_p_mw = 100
net.gen.min_p_mw = 0

# maximum line and transformer loading percent determine the convergence
# Set maximum line loading percentage
net.line.max_loading_percent = 10000

# Set transformer loading percent as unlimited
net.trafo.max_loading_percent = 1e20

# Set min_p_mw at slack bus
net.ext_grid.min_p_mw = 0 

# Set the sgen NOT in service
net.sgen.in_service = False

# Set maximum generation capacity
net.gen.max_p_mw = 500

# maximum line and transformer loading percent determine the convergence
# Set maximum line loading percentage
net.line.max_loading_percent = 400
# Set transformer loading percent as unlimited
net.trafo.max_loading_percent = 1e20

# Set min_p_mw at slack bus
net.ext_grid.min_p_mw = 0

pp.rundcopp(net)

/opt/homebrew/lib/python3.11/site-packages/pandapower/io_utils.py:509: FutureWarning: Passing literal json to 'read_json' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  df = pd.read_json(self.obj, precise_float=True, convert_axes=False, **self.d)
/opt/homebrew/lib/python3.11/site-packages/pandapower/io_utils.py:509: FutureWarning: Passing literal json to 'read_json' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  df = pd.read_json(self.obj, precise_float=True, convert_axes=False, **self.d)
/opt/homebrew/lib/python3.11/site-packages/pandapower/io_utils.py:509: FutureWarning: Passing literal json to 'read_json' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  df = pd.read_json(self.obj, precise_float=True, convert_axes=False, **self.d)
/opt/homebrew/lib/python3.11/site-packag

In [16]:
pp.to_excel(net, 'network/case1354_original_network.xlsx')

In [17]:
# Change bus name from 'Name' to 'Bus_no'
net.bus.name.name = 'Bus_no'
# Bus no. starts from 0
net_index = net.bus.name.astype(int) - 1

In [18]:
# Create DataFrame to store network data
network = pd.DataFrame()

# Set network index
network.index = net_index

In [19]:
############################################################################################################

# trouble shooting issue with mismatched buses between network and poly_cost_index

############################################################################################################



# Read poly_cost_index
poly_cost_index = pd.read_csv('./network/poly_cost_index.csv', index_col=0, header=0)

# Assuming net.poly_cost is already loaded
# Filter net.poly_cost to have exactly 54 rows
net.poly_cost = net.poly_cost.iloc[:54]

# Check the shapes
print("poly_cost_index shape:", poly_cost_index.shape)
print("Adjusted net.poly_cost shape:", net.poly_cost.shape)

# Reshape poly_cost_index and check the length
reshaped_index = poly_cost_index.values.reshape((-1))
print("Reshaped index length:", len(reshaped_index))

# Ensure lengths match before assignment
if len(reshaped_index) == net.poly_cost.shape[0]:
    net.poly_cost.index = reshaped_index
    print("Index assignment successful.")
else:
    print(f"Length mismatch: Expected axis has {net.poly_cost.shape[0]} elements, new values have {len(reshaped_index)} elements")

print(poly_cost_index.shape)
print(net.poly_cost.shape)


poly_cost_index shape: (54, 1)
Adjusted net.poly_cost shape: (54, 8)
Reshaped index length: 54
Index assignment successful.
(54, 1)
(54, 8)


In [20]:
# Get features from 'bus
network['vn_kv'] = net.bus.vn_kv.values

# Get features from 'load'
net.load.index = net.load.bus
network['load_p_mw'] = net.load.p_mw

# Get features from 'gen'
net.gen.index = net.gen.bus
network['max_gen_p_mw'] = net.gen.max_p_mw
network['min_gen_p_mw'] = net.gen.min_p_mw

# Get features from 'ext_grid' (ext_grid bus no: 68)
net.ext_grid.index = net.ext_grid.bus
network['max_gen_p_mw'][net.ext_grid.index.values] = net.ext_grid.max_p_mw
network['min_gen_p_mw'][net.ext_grid.index.values] = net.ext_grid.min_p_mw

# Get features from 'poly_cost'
# This needs manual work
poly_cost_index = pd.read_csv('./network/poly_cost_index.csv', index_col=0, header=0)
net.poly_cost.index = poly_cost_index.values.reshape((-1))
network['price_1'] = net.poly_cost.cp1_eur_per_mw
network['price_2'] = net.poly_cost.cp2_eur_per_mw2

# Additional features

network['x_pos'] = net.bus_geodata.x
network['y_pos'] = net.bus_geodata.y

/var/folders/0t/9z26630n32l3qw5w80zvvjqw0000gn/T/ipykernel_11465/111530940.py:15: FutureWarning: ChainedAssignmentError: behaviour will change in pandas 3.0!
You are setting values through chained assignment. Currently this works in certain cases, but when using Copy-on-Write (which will become the default behaviour in pandas 3.0) this will never work to update the original DataFrame or Series, because the intermediate object on which we are setting values will behave as a copy.
A typical example is when you are setting values in a column of a DataFrame, like:

df["col"][row_indexer] = value

Use `df.loc[row_indexer, "col"] = values` instead, to perform the assignment in a single step and ensure this keeps updating the original `df`.

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

  network['max_gen_p_mw'][net.ext_grid.index.values] = net.ext_grid.max_p_mw
/var/folders/0t/9z26630n32l3qw5w80zvvj

In [21]:
branch = net.line[['from_bus', 'to_bus']]
branch

,from_bus,to_bus
0,1073,801
1,651,1102
2,855,591
3,976,879
4,976,1006
5,746,1329
6,431,1329
7,204,1329
8,269,1329
9,1205,1329


In [22]:
## line values are off c_nf


branch_attr = net.line[['c_nf_per_km', 'r_ohm_per_km', 'x_ohm_per_km']]
branch_attr

,c_nf_per_km,r_ohm_per_km,x_ohm_per_km
0,0.0,0.087120,0.378004
1,0.0,1.433124,8.077960
2,0.0,0.271040,2.090880
3,0.0,0.288800,3.552240
4,0.0,0.433200,5.443880
5,0.0,0.067760,0.275880
6,0.0,0.062920,0.352836
7,0.0,0.058080,0.281204
8,0.0,0.053240,0.309760
9,0.0,0.091960,0.271040


In [23]:

trafo = net.trafo[['hv_bus', 'lv_bus']]
trafo

,hv_bus,lv_bus
0,1348,355
1,1348,355
2,15,355
3,470,1127
4,1059,731
5,1059,731
6,111,675
7,111,675
8,510,887
9,510,887


In [24]:
trafo_attr = net.trafo[['sn_mva', 'tap_neutral', 'tap_pos', 'tap_step_percent', 'vk_percent']]
trafo_attr

,sn_mva,tap_neutral,tap_pos,tap_step_percent,vk_percent
0,99.999,0.0,-1.0,1.7857,4.910243
1,591.000,0.0,-1.0,6.3830,14.713259
2,591.000,0.0,-1.0,2.0773,14.405467
3,591.000,0.0,-1.0,6.3830,13.980442
4,591.000,0.0,-1.0,6.3830,15.174458
5,591.000,0.0,-1.0,6.3830,17.338768
6,99.999,0.0,-1.0,1.7857,5.210757
7,99.999,0.0,-1.0,1.7857,4.353732
8,591.000,0.0,-1.0,6.3830,16.734663
9,591.000,0.0,-1.0,6.3830,15.366595


In [25]:
network.to_csv('network/node_features.csv')
branch.to_csv('network/branch_index.csv')
branch_attr.to_csv('network/branch_attr.csv')
trafo.to_csv('network/trafo_index.csv')
trafo_attr.to_csv('network/trafo_attr.csv')

In [26]:
from sklearn.preprocessing import MinMaxScaler
import joblib

# Normalize branch atrributes
branch_attr_scaler = MinMaxScaler()
branch_attr_scaled = branch_attr_scaler.fit_transform(branch_attr)
pd.DataFrame(branch_attr_scaled).to_csv('network/branch_attr_scaled.csv')
joblib.dump(branch_attr_scaler, 'network/branch_attr_scaler.scaler')

# Normalize transformer attributes
trafo_attr_scaler = MinMaxScaler()
trafo_attr_scaled = trafo_attr_scaler.fit_transform(trafo_attr)
pd.DataFrame(trafo_attr_scaled).to_csv('network/trafo_attr_scaled.csv')
joblib.dump(trafo_attr_scaler, 'network/trafo_attr_scaler.scaler')

['network/trafo_attr_scaler.scaler']